In [7]:
pip install git+https://github.com/tqsd/special_issue_quantum.git@master

  Cloning https://github.com/tqsd/special_issue_quantum.git (to revision master) to /tmp/pip-req-build-6exoh2fw
  Running command git clone --filter=blob:none --quiet https://github.com/tqsd/special_issue_quantum.git /tmp/pip-req-build-6exoh2fw
  Resolved https://github.com/tqsd/special_issue_quantum.git to commit 5931b967e2fa4dbc28201c9480c02554e4a9f81b
  Preparing metadata (setup.py) ... done


In [8]:
# Imports
from IPython.display import display

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.special import factorial
import json
import uuid

from qsi.coordinator import Coordinator
from qsi.state import State, StateProp

plt.style.use('paper.mplstyle')

OSError: 'paper.mplstyle' is not a valid package style, path of style file, URL of style file, or library style name (library styles are listed in `style.available`)

In [11]:
def photon_number_projectors(dim_e,dim_l):
    ''' Returns a photon number projector to be applied to the early and late states in the joint e x l space.

     Args:
        dim_e (int): Dimension of the early state
        dim_l (int): Dimension of the late state

    Returns:
        np.ndarray: The number operator on the early state
        np.ndarray: The number operator on the late state

    '''
    a_e  = np.zeros((dim_e, dim_e))
    for i in range(0,dim_e-1):
        a_e[i,i+1] = np.sqrt(i+1)
    a_dagger_e = a_e.conj().T
    n_hat_e = a_dagger_e@a_e

    a_l  = np.zeros((dim_l, dim_l))
    for i in range(0,dim_l-1):
        a_l[i,i+1] = np.sqrt(i+1)
    a_dagger_l = a_l.conj().T
    n_hat_l = a_dagger_l@a_l

    n_hat_e = np.kron(n_hat_e, np.eye(dim_l))
    n_hat_l = np.kron(np.eye(dim_e), n_hat_l)

    return n_hat_e, n_hat_l


In [12]:
#start the coordinator
coordinator = Coordinator(port=20002)

#Add the memory and the coherent source
mem = coordinator.register_component(module="memory.py", runtime="python")
cs = coordinator.register_component(module="coherent_source.py", runtime = "python")

# Run the coordinator process
coordinator.run()

Exception in thread Thread-22 (handle_connections):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/qsi/socket_handler.py", line 36, in handle_connections
    s.bind(('localhost', port))
OSError: [Errno 98] Address already in use


[stderr] /usr/bin/python3: can't open file '/content/memory.py': [Errno 2] No such file or directory
[stderr] /usr/bin/python3: can't open file '/content/coherent_source.py': [Errno 2] No such file or directory


KeyboardInterrupt: 

In [13]:
#define the memory paramters
storage_time = 0e-6            #s
mem_type = 'Lambda895'
trunc = 5

In [14]:
# Set the memory paramters.
mem.set_param("storage_time", storage_time)
mem.set_param("memory_type", mem_type)
mem.set_param("memory_truncation",int(trunc))
mem.send_params()

# Initialise the internal state of the memory
mem_state_bs = mem.state_init()[0]

# Set the coherent source paramters
cs.set_param("alpha", 1+0j)
cs.send_params()


# define properties of the input state (early state)
state_early_prop = StateProp(
    state_type="light",
    truncation=trunc,
    wavelength=895,
    polarization='H',
    uuid="early",
    bandwidth =0.5e9

)
# make the early state (vaccum state)
state_early = State(state_early_prop)

# Generate coherent source operators
response, operators = cs.channel_query(
state_early, {"input": state_early_prop.uuid}
)

# apply coherent source operators
state_early.apply_kraus_operators(operators, [state_early_prop])

# create a joint state, consisting of just the early state
whole_state =  state_early

# Add the internal state of the memory to make a joint state e x s
whole_state.join(mem_state_bs)

# Generate the memory storage operators.
response, operators = mem.channel_query(
    whole_state, {"input": state_early_prop.uuid, 'op_type': 'storage'}
)

# Apply memory storage operators.
whole_state.apply_kraus_operators(operators, whole_state.get_all_props(response["kraus_state_indices"]))


# define the properties of the late state
state_late_prop = StateProp(
    state_type="light",
    truncation=trunc,
    wavelength=895,
    polarization='H',
    uuid="late",
    bandwidth =0.5e9

)
# create the late state
state_late = State(state_late_prop)

# join the late state to the e x s state - e x s x l
whole_state.join( state_late )

# generate the retrieval operators
response, operators = mem.channel_query(
   whole_state, {"input": state_late_prop.uuid, 'op_type': 'retrieval'}
)

# apply the retrieval opertors
whole_state.apply_kraus_operators(operators, whole_state.get_all_props(response["kraus_state_indices"]))


# create the \hat{n} projectors working on the e x l space
n_hat_e, n_hat_l = photon_number_projectors(state_early_prop.truncation,state_late_prop.truncation )

# trace out the storage state leaving just e x l
red = whole_state.get_reduced_state([state_early_prop,state_late_prop])

# apply the number operators to early and late states.
photon_num_e = np.trace(red @ n_hat_e)
photon_num_l =  np.trace(red @ n_hat_l)

KeyboardInterrupt: 

In [ ]:
photon_num_e, photon_num_l

# Using a test memory
Below shows the initalisation of an example test memory, which we use to generate a truncation figure.

In [ ]:
def truncation_scan(truncat, coherent_state = True ):
    storage_time = 0e6

    mem.set_param("storage_time", storage_time)
    mem.set_param("memory_type", 'Test')
    mem.set_param("memory_truncation", int(truncat))

    mem.set_param("t_in", np.sqrt(0.5))             # \eta_{read_in} = sqrt(0.5)
    mem.set_param("t_out", np.sqrt(0.5))            # \eta_{read_out} = sqrt(0.5)
    mem.set_param("k_e", 0.5)              # \kappa_e = 0.5
    mem.set_param("k_l", 0.5)              # \kappa_l = 0.5
    mem.set_param("nB_e",  1)            # n_b_e = 0.5
    mem.set_param("nB_l", 1)             # n_b_l = 0.5
    mem.send_params()


    cs.set_param("alpha", 1+0j)
    cs.send_params()

    mem_state_bs = mem.state_init()[0]

    state_early_prop = StateProp(
        state_type="light",
        truncation=truncat,
        wavelength= 895,
        polarization= 'H',
        uuid="early",
        bandwidth = 0.5e9

    )
    state_early = State(state_early_prop)

    # switch between coherent_state and single photon input.
    if coherent_state:
        response, operators = cs.channel_query(
        state_early, {"input": state_early_prop.uuid}
        )
        state_early.apply_kraus_operators(operators, [state_early_prop])

    else:
        state_early.state[0,0]  = 0
        state_early.state[1,1]  = 1

    whole_state =  state_early
    whole_state.join(mem_state_bs)

    response, operators = mem.channel_query(
        whole_state, {"input": state_early_prop.uuid, 'op_type': 'storage'}
    )
    whole_state.apply_kraus_operators(operators, whole_state.get_all_props(response["kraus_state_indices"]))


    state_late_prop = StateProp(
        state_type="light",
        truncation=truncat,
        wavelength= 895,
        polarization= 'H',
        uuid="late",
        bandwidth = 0.5e9

    )

    state_late = State(state_late_prop)
    whole_state.join( state_late )


    response, operators = mem.channel_query(
       whole_state, {"input": state_late_prop.uuid, 'op_type': 'retrieval'}
    )
    whole_state.apply_kraus_operators(operators, whole_state.get_all_props(response["kraus_state_indices"]))


    n_hat_e, n_hat_l = photon_number_projectors(state_early_prop.truncation,state_late_prop.truncation )

    red = whole_state.get_reduced_state([state_early_prop,state_late_prop])

    photon_no_e = np.trace(red @ n_hat_e)
    photon_no_l =  np.trace(red @ n_hat_l)
    return photon_no_e, photon_no_l


In [ ]:
truncation = np.arange(2,9)
coherent = np.zeros((truncation.shape[0],2))
single =  np.zeros((truncation.shape[0],2))

for i in range(truncation.shape[0]):
    single[i] = truncation_scan(truncation[i], coherent_state = False)
    coherent[i] = truncation_scan(truncation[i], coherent_state = True)

In [ ]:
plt.plot(truncation, single[:,1], label = 'Single photon')
plt.plot(truncation, coherent[:,1], label = 'Coherent state')
plt.legend()
plt.ylabel(r"$\langle \hat{n} \rangle_l$")
plt.xlabel("Truncation")
plt.ylim(0,1)
